In [0]:
from pyspark.sql.functions import col, cast
from pyspark.sql.types import *

# Event Hub details
EH_CONN_STR = "Endpoint=sb://eventhub-namespace-twitter-analysis.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=IUL8Owc4gtWYo5ITUXoMMa0ZeDZvQALWe+AEhLNTAlo="
EH_NAMESPACE = "eventhub-namespace-twitter-analysis"  # my-event-hubs-namespace
EH_KAFKA_TOPIC = "eventhub-twitter-analysis"  # my-event-hub
EH_BOOTSTRAP_SERVERS = f"{EH_NAMESPACE}.servicebus.windows.net:9093"
EH_SASL_WRITE = f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{EH_CONN_STR}";'

# Azure SQL Database Details
SQL_DB_URL = "jdbc:sqlserver://sqlserver-twitter-analysis.database.windows.net:1433;database=db-twitter-analysis"
SQL_DB_USERNAME = "user007"
SQL_DB_PASSWORD = "RandomPassword10503040"


# standard configuration options
topic_name = EH_KAFKA_TOPIC
eh_namespace_name = EH_NAMESPACE
eh_sasl = EH_SASL_WRITE
bootstrap_servers = EH_BOOTSTRAP_SERVERS
kafka_options = {
    "kafka.bootstrap.servers": bootstrap_servers,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.request.timeout.ms": "60000",
    "kafka.session.timeout.ms": "30000",
    "startingOffsets": "earliest",
    "kafka.sasl.jaas.config": eh_sasl,
    "subscribe": topic_name,
}

In [0]:
%sql

CREATE TABLE IF NOT EXISTS tweets
(
    Key BINARY,
    Value BINARY,
    Topic STRING,
    Partition INT,
    Offset LONG,
    Timestamp TIMESTAMP,
    TimestampType INT,
    Value_text STRING
) using DELTA

In [0]:
kafka_df = (
    spark.readStream.format("kafka")
    .options(**kafka_options)
    .load()
    .withColumn("value_text", col("value").cast("string"))
)

In [0]:
kafka_df.writeStream.outputMode("append").option(
    "checkpointLocation", "/tmp/delta/events/_checkpoints/"
).toTable("tweets")

In [0]:
%sql
SELECT
  value_text
FROM
  hive_metastore.default.tweets;

value_text
id_str
created_at
text
id_str
created_at
text
"""{\""id_str\"":275821969415483393,\""created_at\"":\""2012-12-04 04:40:56\"",\""text\"":\""b'Am happy to report that Tesla was narrowly cash flow positive last week. Continued improvement expected through year end.'\""}"""
"""{\""id_str\"":275346615634624514,\""created_at\"":\""2012-12-02 21:12:03\"",\""text\"":\""b'Uranium ore now available on Amazon! So much more convenient than Uzbekistan #AwesomeXmasGifts http:\\/\\/t.co\\/jqWAJrnY'\""}"""
"""{\""id_str\"":274090388669734912,\""created_at\"":\""2012-11-29 10:00:15\"",\""text\"":\""b'@shervin Thanks Shervin!'\""}"""
"""{\""id_str\"":273671124909432833,\""created_at\"":\""2012-11-28 06:14:15\"",\""text\"":\""b'RT @TheEconomist: Mexico is changing in ways that will profoundly affect America http:\\/\\/t.co\\/icj9xRYE'\""}"""


In [0]:
from pyspark.sql.functions import (
    col,
    current_timestamp,
    from_json,
    cast,
    lit,
    from_unixtime,
    to_timestamp,
    sum,
    avg,
    when,
    count,
    round,
    max,
)
from pyspark.sql.types import (
    StructField,
    StructType,
    IntegerType,
    StringType,
    DoubleType,
    LongType,
    TimestampType,
)

tweets_schema = StructType(
    fields=[
        StructField("id_str", StringType(), False),
        StructField("created_at", TimestampType(), False),
        StructField("text", StringType(), True),
    ]
)

In [0]:
tweets_df = (
    spark.sql("SELECT * FROM tweets")
    .drop("key", "value", "partition", "offset", "Timestamp", "Timestamptype")
    .withColumn("JSON", from_json(col("value_text"), tweets_schema))
    .withColumn("id", col("JSON.id_str"))
    .withColumn("text", col("JSON.text"))
    .withColumn("created_at", col("JSON.created_at"))
    .drop("JSON", "value_text")
    .dropDuplicates()
)

In [0]:
display(tweets_df)

Topic,id,text,created_at
eventhub-twitter-analysis,716356456593756160,"b'@grebakshay We are working on an exact date, but Tesla will be in India before 3 production starts.'",2016-04-02T20:07:43Z
eventhub-twitter-analysis,716347450148040704,"b'@vicentes Roadster, S and X customers are the ones who got us here and are paying for 3. They deserve a lot of credit.'",2016-04-02T19:31:55Z
eventhub-twitter-analysis,825036546206076928,"b""@suzie_official Yeah, am hearing this from a lot of people & it's getting me down. I'm just trying to make a positi\xe2\x80\xa6 https://t.co/IN6Sqs586S""",2017-01-27T17:43:35Z
eventhub-twitter-analysis,716356220156686336,b'@Model3Owners They will. We spent a lot of time on those.',2016-04-02T20:06:46Z
eventhub-twitter-analysis,228659334513573888,"b""@iproducefodrake Asimov's Foundation points out that all civilizations fall. Must ensure dark period is short & finite.""",2012-07-27T01:13:28Z
eventhub-twitter-analysis,228690043949158401,"b'@nopev Biggest risk is that we take too long to solve the sustainable energy problem, causing climate change meltdown.'",2012-07-27T03:15:29Z
eventhub-twitter-analysis,228695905338019842,"b'@walalm Yes, my absolute goal w Tesla has always been mass market electric cars. Will finally get there with Gen3 program.'",2012-07-27T03:38:47Z
eventhub-twitter-analysis,824942405262544896,b'RT @Hyperloop: Teams arrive today for Testing Week before first-ever Hyperloop Competition at @SpaceX HQ on Jan. 27-29 \xe2\x86\x92 https://t.co/MHrFC\xe2\x80\xa6',2017-01-27T11:29:30Z
eventhub-twitter-analysis,228677717632098304,"b""Compared to past, today's world is fantastic & likely will be for many decades. Just need to cover future downside risk.""",2012-07-27T02:26:31Z
eventhub-twitter-analysis,716696932073152512,"b'@mindthreat @RyanHummer Yes, if the trend continues, before'",2016-04-03T18:40:38Z


In [0]:
try:
    (
        tweets_df.coalesce(1)
        .write.format("jdbc")
        .option("url", SQL_DB_URL)
        .option("dbtable", "dbo.tweets")
        .option("user", SQL_DB_USERNAME)
        .option("password", SQL_DB_PASSWORD)
        .mode("append")
        .save()
    )

    print("Successfully write data into target SQL database")
except Exception as error:
    print("An exception occurred:", error)

An exception occurred: An error occurred while calling o882.save.
: com.microsoft.sqlserver.jdbc.SQLServerException: Cannot open server 'sqlserver-twitter-analysis' requested by the login. Client with IP address '20.28.9.255' is not allowed to access the server.  To enable access, use the Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. ClientConnectionId:1524b518-6f0e-4d3a-995c-cacae0341fe4
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:265)
	at com.microsoft.sqlserver.jdbc.TDSTokenHandler.onEOF(tdsparser.java:300)
	at com.microsoft.sqlserver.jdbc.TDSParser.parse(tdsparser.java:133)
	at com.microsoft.sqlserver.jdbc.TDSParser.parse(tdsparser.java:37)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.sendLogon(SQLServerConnection.java:6299)
	at com.microsoft.sqlserver.jdbc.SQLServer